### Deploy Web App on Azure Container Services (ACS)
In this notebook we will set up an Azure Container Service which will be managed by DC/OS. We will then take the Docker image we created earlier that contains our app and deploy it to the ACS cluster. Then we will check everything is working by sending an image to it and getting it scored.

The process is split into the following steps:
* [Define our resource names](#section1)
* [Login to Azure](#section2)
* [Create the ACS](#section3)
* [Create a tunnel to the head node](#section4)
* [Create a JSON schema of our APP and push it to the cluster](#section5)
* [Test our app](TestWebApp.ipynb)
* [Tear it all down](#section7)

This guide assumes is designed to be run on linux and requires that the Azure CLI is installed.

<a id='section1'></a>

## Setup
Below are the various name definitions for the resources needed to setup ACS as well as the name of the Docker image we will be using.

**Some outputs (and inputs) below have been hidden/masked for confidentiality**

In [140]:
resource_group = "m*****g" # Feel free to modify these
acs_name = "m*****s"
location = "s*****s"

image_name = 'masalvar/cntkresnet' 
selected_subscription = "'Azure *****'" # If you have multiple subscriptions select 
                                        # the subscription you want to use here

<a id='section2'></a>

## Azure account login
The command below will initiate a login to your Azure account. It will pop up with an url to go to where you will enter a one off code and log into your Azure account using your browser.

In [ ]:
!az login -o table

In [141]:
!az account set --subscription $selected_subscription

In [ ]:
!az account show

<a id='section3'></a>

## Create resources and dependencies

### Create resource group
Azure encourages the use of groups to organise all the Azure components you deploy. That way it is easier to find them but also we can deleted a number of resources simply by deleting the group.

In [ ]:
!az group create --name $resource_group --location $location

### Create ssh
Create ssh key if one not present. This is needed for the tunnel we will be creating to the head node in order to interact with Marathon.

In [ ]:
import os
if not os.path.exists('{}/.ssh/id_rsa'.format(os.environ['HOME'])):
    !ssh-keygen -t rsa -b 2048 -N "" -f ~/.ssh/id_rsa

### Create ACS
We are going to deploy a small pool of 2 Standard D2 VMs. Each VM has 2 cores and 7 GB of RAM. This is the default choice when setting up an ACS cluster. 

In [145]:
json_data=!az acs create --name $acs_name --resource-group $resource_group --admin-username mat --dns-prefix $acs_name --agent-count 2

In [146]:
json_dict = json.loads(''.join(json_data))

In [147]:
if json_dict['properties']['provisioningState'] == 'Succeeded':
    print('Succensfully provisioned ACS {}'.format(acs_name))
    _,ssh_addr,_,_,ssh_port, = json_dict['properties']['outputs']['sshMaster0']['value'].split()

Succensfully provisioned ACS mscntkacs


We can check to see if our ACS cluster was succesuflly provisioned with the command below.

In [ ]:
!az acs list --resource-group $resource_group --output table

With the below command we can check various of the properties of our ACS cluster.

In [ ]:
!az acs show --name $acs_name --resource-group $resource_group

From the JSON above we can see that we indeed have a cluster of two D2 VMs

<a id='section4'></a>

## Create SSH tunnel
Create ssh tunnel from dsvm to ACS cluster management

In [151]:
%%bash --bg -s $ssh_port $ssh_addr
ssh -o StrictHostKeyChecking=no -fNL 1212:localhost:80 -p $1 $2

Starting job # 7 in a separate thread.


<a id='section5'></a>

## Marathon deployment
Below we create a JSON schema of our application which we will then pass to marathon. Using this schema Marathon will spin up our application in ACS.

In [152]:
application_id = "/cntkresnet"

In [153]:
app_template = {
  "id": application_id,
  "cmd": None,
  "cpus": 1,
  "mem": 1024,
  "disk": 100,
  "instances": 1,
  "acceptedResourceRoles": [
    "slave_public"
  ],
  "container": {
    "type": "DOCKER",
    "volumes": [],
    "docker": {
      "image": image_name,
      "network": "BRIDGE",
      "portMappings": [
        {
          "containerPort": 88,
          "hostPort": 80,
          "protocol": "tcp",
          "name": "80",
          "labels": {}
        }
      ],
      "privileged": False,
      "parameters": [],
      "forcePullImage": True
    }
  },
  "healthChecks": [
    {
      "path": "/",
      "protocol": "HTTP",
      "portIndex": 0,
      "gracePeriodSeconds": 300,
      "intervalSeconds": 60,
      "timeoutSeconds": 20,
      "maxConsecutiveFailures": 3
    }
  ]
}

In [154]:
def write_json_to_file(json_dict, filename):
    with open(filename, 'w') as outfile:
        json.dump(json_dict, outfile)

In [155]:
write_json_to_file(app_template, 'marathon.json')

In [156]:
!curl -X POST http://localhost:1212/marathon/v2/apps -d @marathon.json -H "Content-type: application/json"

{"id":"/cntkresnet","cmd":null,"args":null,"user":null,"env":{},"instances":1,"cpus":1,"mem":1024,"disk":100,"gpus":0,"executor":"","constraints":[],"uris":[],"fetch":[],"storeUrls":[],"backoffSeconds":1,"backoffFactor":1.15,"maxLaunchDelaySeconds":3600,"container":{"type":"DOCKER","volumes":[],"docker":{"image":"masalvar/cntkresnet","network":"BRIDGE","portMappings":[{"containerPort":88,"hostPort":80,"servicePort":0,"protocol":"tcp","name":"80","labels":{}}],"privileged":false,"parameters":[],"forcePullImage":true}},"healthChecks":[{"path":"/","protocol":"HTTP","portIndex":0,"gracePeriodSeconds":300,"intervalSeconds":60,"timeoutSeconds":20,"maxConsecutiveFailures":3,"ignoreHttp1xx":false}],"readinessChecks":[],"dependencies":[],"upgradeStrategy":{"minimumHealthCapacity":1,"maximumOverCapacity":1},"labels":{},"acceptedResourceRoles":["slave_public"],"ipAddress":null,"version":"2017-04-16T13:49:11.106Z","residency":null,"secrets":{},"taskKillGracePeriodSeconds":null,"ports":[0],"portDef

In [157]:
from time import sleep
for i in range(20):
    json_data = !curl http://localhost:1212/marathon/v2/apps
    if json.loads(json_data[-1])['apps'][0]['tasksRunning']==1:
        print('Web app ready')
        break
    else:
        print('Preparing Web app')
    sleep(10)
else:
    print('Timeout! Something went wrong!')

Preparing Web app
Preparing Web app
Preparing Web app
Preparing Web app
Preparing Web app
Preparing Web app
Preparing Web app
Preparing Web app
Preparing Web app
Preparing Web app
Preparing Web app
Preparing Web app
Preparing Web app
Preparing Web app
Preparing Web app
Preparing Web app
Web app ready


You can also monitor the deployment of the application by pointing your browser at http://localhost:1200/#/services/  
For further notes on the Marathon REST API look [here](https://mesosphere.github.io/marathon/docs/rest-api.html)

In [158]:
app_url = json_dict['properties']['outputs']['agentFQDN']['value']

In [160]:
print('Application URL: {}'.format(app_url))
print('Application ID: {}'.format(application_id))

Application URL: mscntkacsagents.southcentralus.cloudapp.azure.com
Application ID: /cntkresnet


Once it is all set up you can test the app out in the [Test Web App](TestWebApp.ipynb) notebook.

<a id='section7'></a>

## Tear it all down 
Once you are done with your ACS you can use the following two commands to destroy it all.

In [161]:
!az acs delete --resource-group $resource_group --name $acs_name

In [162]:
!az group delete --name $resource_group -y